In [47]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from sklearn.model_selection import train_test_split

In [48]:
PATH_CSV_LABELS = '../tabellaPseudoLabelsDatasetCompleto.csv'
PATH_DATASET_LAVORATO = '../../Nuovi Dati/TemplateGeneratiClean'
PATH_MODELLO = '../../Modelli-v2/modello_pesi_inizializzati_speckle'
PATH_MODELLO_FINALE_RIADDESTRATO = '../../Modelli-v2/modello_finale_riaddestrato_speckle'

N_IMG = 1
FORMATO_IMG = 'jpg'
CHANNELS = 1
HEIGHT = 512
WIDTH = 512
TARGET_SIZE = (WIDTH, HEIGHT)

HUBER_DELTA = 1.0

In [49]:
df_labels = pd.read_csv(PATH_CSV_LABELS)
df_labels.head(5)

,Acquisizione,Label,Label_norm
0,AgatielloRoberto_000,0.648876,50.428711
1,AgatielloRoberto_001,0.759503,61.300294
2,AgatielloRoberto_002,0.747500,60.120716
3,AgatielloRoberto_003,0.818899,67.137182
4,AgatielloRoberto_004,0.825895,67.824714


In [50]:
X = []
y = []

for id_acquisizione in df_labels.Acquisizione:
    path_img = os.path.join(PATH_DATASET_LAVORATO, id_acquisizione, 'Immagini', 'immagine1.jpg')
    image = Image.open(path_img)
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    
    pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
    X.append(normalized_image_array)
    y.append(pseudo_label)
    
X = np.array(X)
y = np.array(y)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [52]:
print(f'X_train: {len(X_train)}')
print(f'X_test: {len(X_test)}')

X_train: 574
X_test: 144


In [53]:
model = keras.models.load_model(PATH_MODELLO)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 255, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2080800)           0         
                                                                 
 dense (Dense)               (None, 64)                133171264 
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 133,171,649
Trainable params: 133,171,649
Non-trainable params: 0
__________________________________________

In [54]:
# model.fit(x=X_train, y=y_train, batch_size=1, epochs=20, validation_data=(X_cv, y_cv))

In [55]:
# model.save(PATH_MODELLO_FINALE)

## Addestramento nuovo modello con Huber loss

In [56]:
conv1_weights = model.layers[0].weights
dense1_weights = model.layers[3].weights
dense2_weights = model.layers[4].weights

new_model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS), weights=conv1_weights),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu', weights=dense1_weights),
    layers.Dense(1, weights=dense2_weights)
])

In [57]:
new_model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=HUBER_DELTA), metrics=['mae'])

In [58]:
new_model.fit(x=X_train, y=y_train, batch_size=1, epochs=10)

Epoch 1/10


2023-07-25 17:25:24.262114: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


574/574 [==============================] - 65s 111ms/step - loss: 16.9230 - mae: 17.4139
Epoch 2/10
574/574 [==============================] - 63s 110ms/step - loss: 14.8870 - mae: 15.3796
Epoch 3/10
574/574 [==============================] - 64s 111ms/step - loss: 12.6583 - mae: 13.1497
Epoch 4/10
574/574 [==============================] - 68s 118ms/step - loss: 9.2616 - mae: 9.7478
Epoch 5/10
574/574 [==============================] - 66s 115ms/step - loss: 7.0139 - mae: 7.5004
Epoch 6/10
574/574 [==============================] - 64s 112ms/step - loss: 5.1495 - mae: 5.6323
Epoch 7/10
574/574 [==============================] - 63s 110ms/step - loss: 4.1984 - mae: 4.6750
Epoch 8/10
574/574 [==============================] - 63s 110ms/step - loss: 3.6229 - mae: 4.0946
Epoch 9/10
574/574 [==============================] - 63s 109ms/step - loss: 3.7318 - mae: 4.2045
Epoch 10/10
574/574 [==============================] - 63s 110ms/step - loss: 3.5825 - mae: 4.0617


In [7]:
#new_model.predict(X)

In [6]:
#y

In [59]:
new_model.save(PATH_MODELLO_FINALE_RIADDESTRATO)

INFO:tensorflow:Assets written to: ../../Modelli-v2/modello_finale_riaddestrato_speckle/assets


INFO:tensorflow:Assets written to: ../../Modelli-v2/modello_finale_riaddestrato_speckle/assets


### Import nuovo modello

In [60]:
new_model = tf.keras.models.load_model(PATH_MODELLO_FINALE_RIADDESTRATO)

In [61]:
def avg(lst):
    return sum(lst) / len(lst)

In [64]:
avg(abs(new_model.predict(X_test) - y_test))

5/5 [==============================] - 1s 210ms/step


array([13.83410554])

In [30]:
previsioni_nuovo_modello = new_model.predict(X)

23/23 [==============================] - 4s 157ms/step


In [25]:
previsioni_nuovo_modello[10:20]

array([[60.596878],
       [61.717056],
       [56.647217],
       [61.091373],
       [39.67559 ],
       [55.025955],
       [77.801384],
       [66.342995],
       [46.09707 ],
       [67.43661 ]], dtype=float32)

In [26]:
y_test[10:20]

array([[42.26044055],
       [60.98299622],
       [56.07490024],
       [65.74218707],
       [51.6002194 ],
       [45.35021024],
       [71.2017892 ],
       [74.4499737 ],
       [31.00339434],
       [42.01341109]])